# 如何从预构建的 ReAct 代理返回结构化输出


!!!信息“先决条件”

- [代理架构](../../concepts/agentic_concepts/)
- [聊天模型](https://js.langchain.com/docs/concepts/chat_models/)
- [工具](https://js.langchain.com/docs/concepts/tools/)
- [结构化输出](https://js.langchain.com/docs/concepts/structed_outputs/)


要从预构建的 ReAct 代理返回结构化输出，您可以向 [`createReactAgent`](https://langchain-ai.github.io/langgraphjs/reference/functions/prebuilt.createReactAgent.html) 提供具有所需输出模式的 `responseFormat` 参数：

In [ ]:
import { z } from "zod";
import { createReactAgent } from "@langchain/langgraph/prebuilt";

const responseFormat = z.object({
    // 以这种格式回复用户
    mySpecialOutput: z.string(),
})

const graph = createReactAgent({
    llm: llm,
    tools: tools,
    // 使用“responseFormat”参数指定结构化输出的架构
    responseFormat: responseFormat
})

一旦不再需要进行任何工具调用，代理将通过在对话结束时进行额外的 LLM 调用，以 `responseFormat` 模式指定的格式返回输出。您可以阅读[本指南](/langgraphjs/how-tos/respond-in-format/)以了解另一种方法 - 将结构化输出视为另一种工具 - 从代理实现结构化输出。

## 设置

首先，我们需要安装所需的软件包。
```bash
yarn add @langchain/langgraph @langchain/openai @langchain/core zod
```
本指南将使用 OpenAI 的 GPT-4o 模型。我们可以选择设置 API 密钥
对于 [LangSmith 追踪](https://smith.langchain.com/)，这将为我们提供
一流的可观测性。

In [ ]:
// process.env.OPENAI_API_KEY = "sk_...";

// 可选，在 LangSmith 中添加跟踪
// process.env.LANGSMITH_API_KEY = "ls__..."
process.env.LANGSMITH_TRACING = "true";
process.env.LANGSMITH_PROJECT = "ReAct Agent with system prompt: LangGraphJS";

## 代码

In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import { createReactAgent } from "@langchain/langgraph/prebuilt";
import { tool } from "@langchain/core/tools";
import { z } from "zod";

const weatherTool = tool(
  async (input): Promise<string> => {
    if (input.city === "nyc") {
      return "It might be cloudy in nyc";
    } else if (input.city === "sf") {
      return "It's always sunny in sf";
    } else {
      throw new Error("Unknown city");
    }
  },
  {
    name: "get_weather",
    description: "Use this to get weather information.",
    schema: z.object({
      city: z.enum(["nyc", "sf"]).describe("The city to get weather for"),
    }),
  }
);

const WeatherResponseSchema = z.object({
  conditions: z.string().describe("Weather conditions"),
});

const tools = [weatherTool];

const agent = createReactAgent({
  llm: new ChatOpenAI({ model: "gpt-4o", temperature: 0 }),
  tools: tools,
  responseFormat: WeatherResponseSchema,
}); 

## 用法

现在让我们测试我们的代理：

In [ ]:
const response = await agent.invoke({
  messages: [
    {
      role: "user",
      content: "What's the weather in NYC?",
    },
  ],
})

您可以看到代理输出包含一个 `structuredResponse` 键，其结构化输出符合指定的 `WeatherResponse` 架构，此外还有 `messages` 键下的消息历史记录

In [2]:
response.structuredResponse

{ conditions: 'cloudy' }


### 自定义系统提示

您可能需要进一步自定义第二个 LLM 调用以生成结构化输出并提供系统提示。为此，您可以将带有键 `prompt`、`schema` 的对象传递给 `responseFormat` 参数：

In [ ]:
const agent = createReactAgent({
  llm: new ChatOpenAI({ model: "gpt-4o", temperature: 0 }),
  tools: tools,
  responseFormat: {
    prompt: "Always return capitalized weather conditions",
    schema: WeatherResponseSchema,
  }
}); 

const response = await agent.invoke({
  messages: [
    {
      role: "user",
      content: "What's the weather in NYC?",
    },
  ],
})

您可以验证结构化响应现在是否包含大写值：

In [3]:
response.structuredResponse

{ conditions: 'Cloudy' }
